In [11]:
import pandas as pd
import sqlite3
import os

# --- Ruta base absoluta (ajusta si tu carpeta cambia) ---
RUTA_BASE = r"C:\Users\pablo\App_Pisos\data"

archivo_csv_alquiler = os.path.join(RUTA_BASE, "alquiler.csv")
archivo_csv_ventas = os.path.join(RUTA_BASE, "ventas.csv")
db_alquiler = os.path.join(RUTA_BASE, "alquiler.db")
db_ventas = os.path.join(RUTA_BASE, "ventas.db")

# Columnas esperadas
columnas_esperadas = [
    "titulo", "precio", "ubicacion", "habitaciones", "baños", "metros",
    "superficie_construida", "superficie_util", "tipo_vivienda",
    "descripcion_ampliada", "link", "precio_m2"
]

# Columnas booleanas para tipo de vivienda
lista_tipos_vivienda = ["piso", "atico", "estudio", "apartamento", "duplex", "chalet", "finca", "loft"]

# --- Generar tipo_vivienda desde columnas booleanas ---
def generar_tipo_vivienda(df):
    df = df.copy()
    boolean_matrix = df[lista_tipos_vivienda].fillna(False).astype(bool)
    suma_booleans = boolean_matrix.sum(axis=1)

    conflictivas = df[suma_booleans > 1]
    if not conflictivas.empty:
        print(f"⚠️ Se encontraron {len(conflictivas)} filas con múltiples tipos de vivienda marcados. Se eliminarán.")
        df = df[suma_booleans <= 1]

    df["tipo_vivienda"] = boolean_matrix.to_numpy().argmax(axis=1)
    df["tipo_vivienda"] = df["tipo_vivienda"].apply(lambda val: lista_tipos_vivienda[val])

    return df

# --- Preparar DataFrame ---
def preparar_dataframe(ruta_csv):
    df = pd.read_csv(ruta_csv)

    if all(col in df.columns for col in lista_tipos_vivienda):
        df = generar_tipo_vivienda(df)

    for col in columnas_esperadas:
        if col not in df.columns:
            df[col] = pd.NA

    return df[columnas_esperadas]

# --- Guardar en SQLite ---
def guardar_en_sqlite(df, ruta_sqlite):
    conn = sqlite3.connect(ruta_sqlite)
    df.to_sql("anuncios", conn, if_exists="replace", index=False)
    conn.close()
    print(f"✅ Base de datos guardada en: {ruta_sqlite}")

# --- Ejecutar ---
df_alquiler = preparar_dataframe(archivo_csv_alquiler)
guardar_en_sqlite(df_alquiler, db_alquiler)

df_ventas = preparar_dataframe(archivo_csv_ventas)
guardar_en_sqlite(df_ventas, db_ventas)


✅ Base de datos guardada en: C:\Users\pablo\App_Pisos\data\alquiler.db
✅ Base de datos guardada en: C:\Users\pablo\App_Pisos\data\ventas.db
